# Use a large language model to classify sticky notes based on class exemplars

### Scenario
Imagine you are a team lead and you are taking your team through a reflection activity.  Your team has used a mural to collect thoughts about what things the team should stop doing, continue doing, and start doing.  Now, you need to cluster the sticky notes in the mural by category for further analysis.

Notebook sections:

**Section A - Explore prompting large languge models**
- [Step 1: Set up IBM watsonx.ai foundation model Python library prerequisites](#step1)
- [Step 2: Create a function for prompting a model to classify a message](#step2)

**Section B - Set up your mural**
- [Step 3: Set up MURAL prerequisites](#step3)
- [Step 4: Read class names, exemplars, and feedback messages from your mural](#step4)

**Section C - Classify sticky notes and move them together in the mural**
- [Step 5: Classify sticky notes](#step5)
- [Step 6: Move sticky notes into class boxes in the mural](#step6)

When you run this notebook, your mural will look something like the following image:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-16_classify-by-exemplars_01.png" width="50%" title="Image of a mural" />

# Section A - Explore prompting large languge models

<p>&nbsp;</p>

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-16_classify-by-exemplars_02.png" width="60%" title="Image of a mural" />

<a id="step1"></a>
## Step 1: Set up IBM watsonx.ai foundation model Python library prerequisites
Before you can prompt a foundation model in watsonx.ai, you must perform the following setup tasks:
- 1.1 Create an instance of the Watson Machine Learning service
- 1.2 Associate the Watson Machine Learning instance with the current project
- 1.3 Create an IBM Cloud API key
- 1.4 Create a credentials dictionary for Watson Machine learning
- 1.5 Look up the current project ID

### 1.1 Create an instance of the Watson Machine Learning service
If you don't already have an instance of the IBM Watson Machine Learning service, you can create an instance of the service from the IBM Cloud catalog: <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank">Watson Machine Learning service</a>

### 1.2 Associate an instance of the Watson Machine Learning service with the current project
The _current project_ is the project in which you are running this notebook.

If an instance of Watson Machine Learning is not already associated with the current project, follow the instructions in this topic to do so: <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html?context=wx&audience=wdp" target="_blank">Adding associated services to a project</a>

### 1.3 Create an IBM Cloud API key
Create an IBM Cloud API key by following these instruction: <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui#create_user_key" target="_blank">Creating an IBM Cloud API key</a>

Then paste your new IBM Cloud API key in the code cell below.

In [1]:
g_cloud_apikey = ""

### 1.4 Create a credentials dictionary for Watson Machine learning
See: [Authentication](https://ibm.github.io/watson-machine-learning-sdk/setup_cloud.html#authentication)

In [2]:
# See: https://ibm.github.io/watson-machine-learning-sdk/setup_cloud.html#authentication
g_wml_credentials = { 
    "url"    : "https://us-south.ml.cloud.ibm.com", 
    "apikey" : g_cloud_apikey
}

### 1.5 Look up the current project ID
The _current project_ is the project in which you are running this notebook.  You can get the ID of the current project programmatically by running the following cell.

In [8]:
import os

g_project_id = os.environ["PROJECT_ID"]

<a id="step2"></a>
## Step 2: Create a function for prompting a model to classify sticky notes
- 2.1 Experiment in Prompt Lab
- 2.2 Specify your selected model ID, prompt parameters, and prompt text template
- 2.2 Define a function to perform classification

### 2.1 Experiment in Prompt Lab 
The Prompt Lab in watsonx.ai is a graphical interface for experimenting with prompting foundation models.

Experiment in Prompt Lab to discover what works best:
- Which model returns ideal results
- What parameter settings (eg. decoding) work best
- What prompt text causes the model to respond the way you want

See: [Prompt Lab](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-prompt-lab.html?context=wx&audience=wdp)

### 2.2 Specify your selected model_id, prompt parameters, and prompt text template
In the following three cells, there are example model ID, prompt parameters, and prompt text template you can use.

Replace any of these with values you discovered while experimenting in Prompt Lab.

In [3]:
# Example models:
#
# google/flan-ul2
# google/flan-t5-xxl
# bigscience/mt0-xxl
# eleutherai/gpt-neox-20b
# ibm/granite-13b-instruct-v1

g_model_id = "google/flan-t5-xxl"

In [4]:
g_prompt_parameters = {
    "decoding_method" : "greedy",
    "min_new_tokens"  : 0,
    "max_new_tokens"  : 20
}

**NOTE**

In the following example template, notice the use of `%s` as a placeholder for the class names, few-shot examples, and message text. 

If you replace this template with a prompt you discovered through your experiments in Prompt Lab, remember to include a placeholder for the class names, one for few-shot examples, and one for the message text.

In [5]:
import random
import re

g_prompt_template = """Classify the message into one of three classes: %s

%s

Message: %s
Class: 
"""

def createBasePromptText( exemplars_arr ):
    class_names_arr = []
    exemplars_str = ""
    index_arr = list( range( len( exemplars_arr ) ) )
    random.shuffle( index_arr )
    for i in index_arr:
        class_name = exemplars_arr[i]["class_name"]
        if( class_name not in class_names_arr ):
            class_names_arr.append( class_name )
        exemplars_str += "Message: " + exemplars_arr[i]["message"] + "\nClass: " + class_name + "\n\n"
    class_name_str = ", ".join( class_names_arr )
    exemplars_str = re.sub( r"\s+$", "", exemplars_str )
    return g_prompt_template % ( class_name_str, exemplars_str, "%s" )

In [6]:
g_test_exemplars_arr = [ 
    { "class_name" : "Stop",     "message" : "Our daily stand-ups take too long.  Let's stop doing those." },
    { "class_name" : "Start",    "message" : "I think we should record our meetings so they could be played back" },
    { "class_name" : "Continue", "message" : "I like the \"updates\" Slack channel - very efficient" },
    { "class_name" : "Start",    "message" : "At my friend's company, they have a lottery for plum parking spots.  We should do something similar!" },
    { "class_name" : "Continue", "message" : "Please keep the THINK-Thursday meeting-free afternoon!" },
    { "class_name" : "Stop",     "message" : "Stop having all-hands meetings, because they don't add any value." }
]
g_test_message = "We should take meeting minutes"
g_test_base_prompt_text = createBasePromptText( g_test_exemplars_arr )
g_test_prompt_text = g_test_base_prompt_text % ( g_test_message )
print( g_test_prompt_text )

Classify the message into one of three classes: Stop, Continue, Start

Message: Stop having all-hands meetings, because they don't add any value.
Class: Stop

Message: Please keep the THINK-Thursday meeting-free afternoon!
Class: Continue

Message: I like the "updates" Slack channel - very efficient
Class: Continue

Message: Our daily stand-ups take too long.  Let's stop doing those.
Class: Stop

Message: At my friend's company, they have a lottery for plum parking spots.  We should do something similar!
Class: Start

Message: I think we should record our meetings so they could be played back
Class: Start

Message: We should take meeting minutes
Class: 



### 2.3 Define a function to perform classification
You can prompt foundation models in IBM watsonx.ai programmatically using the foundation models Python library.

See:
- <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-python-lib.html?context=wx&audience=wdp" target="_blank">Introduction to the foundation models Python library</a>
- <a href="https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html" target="_blank">Foundation models Python library reference</a>

Test the library:

In [9]:
from ibm_watson_machine_learning.foundation_models import Model
import json

test_model = Model( g_model_id, g_wml_credentials, g_prompt_parameters, g_project_id )

test_response = test_model.generate( g_test_prompt_text )

print( json.dumps( test_response, indent=3 ) )

{
   "model_id": "google/flan-t5-xxl",
   "created_at": "2023-11-11T02:59:41.349Z",
   "results": [
      {
         "generated_text": "Start",
         "generated_token_count": 2,
         "input_token_count": 165,
         "stop_reason": "eos_token"
      }
   ],
   "system": {
      "warnings": [
         {
            "message": "This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL. URL: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx",
            "id": "disclaimer_warning"
         }
      ]
   }
}


Define a function:

In [10]:
import re

def generate( model_id, prompt_parameters, prompt_text, b_debug=False ):
    model = Model( model_id, g_wml_credentials, prompt_parameters, g_project_id )
    raw_response = model.generate( prompt_text )
    if b_debug:
        print( "\nraw_response:\n" + json.dumps( raw_response, indent=3 ) )
    if ( "results" in raw_response ) \
       and ( len( raw_response["results"] ) > 0 ) \
       and ( "generated_text" in raw_response["results"][0] ):
        return raw_response, raw_response["results"][0]["generated_text"]
    else:
        print( "\nThe model failed to generate an answer" )
        print( "\nDebug info:\n" + json.dumps( raw_response, indent=3 ) )
        return raw_response, ""

def classifyMessage( model_id, prompt_parameters, base_prompt_text, message_text, b_debug=False ):
    prompt_text = base_prompt_text % ( message_text )
    if b_debug:
        print( "Prompt:\n--------------------------START")
        print( prompt_text + "--------------------------END")
    raw_response, generated_output = generate( model_id, prompt_parameters, prompt_text, b_debug )
    class_name = generated_output.strip()
    return class_name

In [11]:
classifyMessage( g_model_id, g_prompt_parameters, g_test_base_prompt_text, g_test_message, b_debug=True )

Prompt:
--------------------------START
Classify the message into one of three classes: Stop, Continue, Start

Message: Stop having all-hands meetings, because they don't add any value.
Class: Stop

Message: Please keep the THINK-Thursday meeting-free afternoon!
Class: Continue

Message: I like the "updates" Slack channel - very efficient
Class: Continue

Message: Our daily stand-ups take too long.  Let's stop doing those.
Class: Stop

Message: At my friend's company, they have a lottery for plum parking spots.  We should do something similar!
Class: Start

Message: I think we should record our meetings so they could be played back
Class: Start

Message: We should take meeting minutes
Class: 
--------------------------END

raw_response:
{
   "model_id": "google/flan-t5-xxl",
   "created_at": "2023-11-11T02:59:58.362Z",
   "results": [
      {
         "generated_text": "Start",
         "generated_token_count": 2,
         "input_token_count": 165,
         "stop_reason": "eos_token"
 

'Start'

# Section B - Set up your mural

<p>&nbsp;</p>

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-16_classify-by-exemplars_03.png" width="50%" title="Image of a mural" />

<a id="step3"></a>
## Step 3: Set up MURAL prerequisites
- 3.1 Create empty, sample mural
- 3.2 Collect mural ID
- 3.3 Get Oauth token
- 3.4 Populate mural

### 3.1 Create empty, sample mural
In the MURAL web interface, create a new, empty mural.

### 3.2 Collect mural ID
You can find the mural ID in the url of a mural.

Mural urls look something like this:

```
https://app.mural.co/t/<workspace>/m/<workspace>/<id>/...
```

What you need to pass to the MURAL API is just after the `/m/`: the \<workspace> and the \<id>.  And you need to join then with a period.

For example, if you have a mural with this url:

```
https://app.mural.co/t/teamideas1234/m/teamideas1234/1234567890123/...
```

Then, the mural ID is: `teamideas1234.1234567890123`

In [11]:
g_mural_id = ""

### 3.3 Collect OAuth token

In [50]:
g_auth_token = ""

### 3.4 Populate mural

In [21]:
import urllib
import json

url = "https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/murals/sample-16_classify-by-exemplars.json"
response = urllib.request.urlopen( url )
encoding = response.info().get_content_charset( "utf8" )
sample_widgets_arr = json.loads( response.read().decode( encoding ) )

In [22]:
import requests
import copy

def putWidget( auth_token, mural_id, widget ):
    # https://developers.mural.co/public/reference/createstickynote
    # https://developers.mural.co/public/reference/createtextbox
    widget_type = "textbox" if ( "text" == widget["type"] ) else widget["type"]
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets/" + re.sub( "\s+", "-", widget_type )
    headers = { "Accept"        : "application/json", 
                "Content-Type"  : "application/json", 
                "Authorization" : "Bearer " + auth_token }
    parms = copy.deepcopy( widget )
    if "id" in parms:
        del parms["id"]
    if "type" in parms:
        del parms["type"]
    response = requests.request( "POST", url, headers = headers, json = parms )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( "[ " + widget["id"] + " ] " + msg )

In [23]:
import time

time.sleep(5)

# Quick!  After running this cell, switch to your browser 
# tab where the mural is to see the change
# ...

for widget in sample_widgets_arr:
    putWidget( g_auth_token, g_mural_id, widget )
print( "Done!" )

Done!


<a id="step4"></a>
## Step 4: Read feedback and exemplars from your mural
- 4.1 Read widgets from the mural
- 4.2 Establish coordinates of class boxes

### 4.1 Read widgets from the mural

In [18]:
def listWidgets( auth_token, mural_id ):
    # https://developers.mural.co/public/reference/getmuralwidgets
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets"
    headers = { "Accept": "application/json", "Authorization": "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None
    if "value" not in response_json:
        print( "No value returned" )
        return None
    return response_json["value"]

In [25]:
g_widgets_arr = listWidgets( g_auth_token, g_mural_id )

In [ ]:
print( json.dumps( g_widgets_arr, indent=3 ) )

### 4.2 Establish coordinates of class boxes
In the mural, there are three boxes, one box for each class: "Stop", "Continue", and "Start".

To be able to move the sticky notes into the correct box by class, we need to know the coordinates of the boxes.

In [43]:
def getWidgetsByType( widgets_arr ):
    boxes_arr = []
    labels_arr = []
    stickies_arr = []
    for widget in widgets_arr:
        obj = { "x" : widget["x"],
                "y" : widget["y"],
                "height" : widget["height"],
                "width"  : widget["width"] }
        if( "shape" == widget["type"] ):
            boxes_arr.append( obj )
        elif( "text" == widget["type"] ):
            obj["id"]       = widget["id"]
            obj["text"]     = re.sub( r"\<[^\>]+\>", "", widget["text"] )
            labels_arr.append( obj )
        elif( "sticky note" == widget["type"] ):
            obj["id"]     = widget["id"]
            obj["text"]   = widget["text"]
            obj["height"] = widget["height"]
            obj["width"]  = widget["width"]
            obj["x_org"]  = widget["x"]
            obj["y_org"]  = widget["y"]
            stickies_arr.append( obj )
    return boxes_arr, labels_arr, stickies_arr

def isInBox( widget, box ):
    widget_left   = widget["x"]
    widget_right  = widget["x"] + widget["width"]
    widget_top    = widget["y"]
    widget_bottom = widget["y"] + widget["height"]
    box_left   = box["x"]
    box_right  = box["x"] + box["width"]
    box_top    = box["y"]
    box_bottom = box["y"] + box["height"]
    if( widget_right >= box_left ) and \
      ( widget_left <= box_right ) and \
      ( widget_bottom >= box_top ) and \
      ( widget_top <= box_bottom ):
        return True
    return False
    
def getContainingBox( widget, boxes_arr ):
    for box in boxes_arr:
        if isInBox( widget, box ):
            return box
    return None
    
def getMuralObjects( widgets_arr ):
    boxes_arr, labels_arr, stickies_arr = getWidgetsByType( widgets_arr )
    for label in labels_arr:
        box = getContainingBox( label, boxes_arr )
        if( box is not None ):
            box["label_id"]       = label["id"]
            box["label_x"]        = label["x"]
            box["label_y"]        = label["y"]
            box["class_name"]     = label["text"]
            box["top_free_space"] = label["y"] + label["height"] + 10
    exemplars_arr = []
    i = 0
    while( i < len( stickies_arr ) ):
        sticky = stickies_arr[i]
        box = getContainingBox( sticky, boxes_arr )
        if( box is not None ):
            exemplars_arr.append( { "class_name" : box["class_name"], "message" : sticky["text"] } )
            new_y = sticky["y"] + sticky["height"] + 10
            if( new_y > box["top_free_space"] ):
                box["top_free_space"] = new_y
            stickies_arr.pop( i )
            continue
        i += 1
    sorted_class_names = [ "Stop", "Continue", "Start" ]
    exemplars_arr = sorted( exemplars_arr, key=lambda d: sorted_class_names.index( d["class_name"] ) )
    print( "Boxes:" )
    for box in boxes_arr:
        print( box["class_name"] )
    print( "\nExemplars:")
    for exemplar in exemplars_arr:
        print( "{:<8}".format( exemplar["class_name"] ) + " - " + exemplar["message"] )
    print( "\nStickies:")
    for sticky in stickies_arr:
        print( sticky["text"] )
    return boxes_arr, exemplars_arr, stickies_arr

In [52]:
g_boxes_arr, g_exemplars_arr, g_stickies_arr = getMuralObjects( g_widgets_arr )

Boxes:
Stop
Continue
Start

Exemplars:
Stop     - Our daily stand-ups take too long. Let's stop doing those.
Stop     - Stop having all-hands meetings, because they don't add any value.
Continue - Please let's continue the THINK-Thursday meeting-free afternoon!
Continue - Loving the on-site day care
Start    - I think we should record our meetings so they could be played back
Start    - We should take meeting minutes

Stickies:
I like the "updates" Slack channel - very efficient
At my friend's company, they have a lottery for plum parking spots.  We should do something similar!
Can we please stop having to sing the national anthem at the start of every day?!
Stop forcing everyone to attend the all-hands in person
The monthly performance awards are a nice recognition
Because of these power outages, we should invest in uninterrupted power supply infrastructure


# Section C - Classify sticky notes and move them together in the mural

<p>&nbsp;</p>

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-16_classify-by-exemplars_01.png" width="50%" title="Image of a mural" />

<a id="step5"></a>
## Step 5: Classify sticky notes
Classify sticky notes based on the class exemplars.

In [46]:
def classifyStickies( exemplars_arr, stickies_arr ):
    base_prompt_text = createBasePromptText( exemplars_arr )
    for sticky in stickies_arr:
        class_name = classifyMessage( g_model_id, g_prompt_parameters, base_prompt_text, sticky["text"] )
        sticky["class_name"] = class_name
    sorted_class_names = [ "Stop", "Continue", "Start" ]
    stickies_arr = sorted( stickies_arr, key=lambda d: sorted_class_names.index( d["class_name"] ) )
    print( "Results:\n" )
    for sticky in stickies_arr:
        class_name = "{:<8}".format( sticky["class_name"] )
        print( class_name + ": " + sticky["text"] )

In [53]:
classifyStickies( g_exemplars_arr, g_stickies_arr )

Results:

Stop    : Can we please stop having to sing the national anthem at the start of every day?!
Stop    : Stop forcing everyone to attend the all-hands in person
Continue: I like the "updates" Slack channel - very efficient
Continue: The monthly performance awards are a nice recognition
Start   : At my friend's company, they have a lottery for plum parking spots.  We should do something similar!
Start   : Because of these power outages, we should invest in uninterrupted power supply infrastructure


<a id="step6"></a>
## Step 6: Move sticky notes into class boxes in the mural

In [48]:
import random

def moveSticky( auth_token, mural_id, sticky_id, new_x, new_y ):
    # https://developers.mural.co/public/reference/updatestickynote
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets/sticky-note/" + sticky_id
    headers = { "Accept"        : "application/json", 
                "Content-Type"  : "application/json", 
                "Authorization" : "Bearer " + auth_token }
    parms = { "x" : new_x, "y" : new_y }
    response = requests.request( "PATCH", url, headers = headers, json = parms )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( "[ " + sticky_id + " ] " + msg )

def moveStickyToBox( auth_token, mural_id, sticky, box ):
    y = box["top_free_space"]
    min_x = box["x"]
    max_x = box["x"] + box["width"] - ( 0.8 * sticky["width"] )
    x = random.uniform( min_x, max_x )
    moveSticky( auth_token, mural_id, sticky["id"], x, y )
    return x, y

def boxesJSON( boxes_arr ):
    boxes_json = {}
    for box in boxes_arr:
        class_name = box["class_name"]
        boxes_json[ class_name ] = box
    return boxes_json

def moveStickiesToClassBoxes( auth_token, mural_id, boxes_arr, stickies_arr ):
    boxes_json = boxesJSON( boxes_arr )
    for sticky in stickies_arr:
        class_name = sticky["class_name"]
        box = boxes_json[ class_name ]
        new_x, new_y = moveStickyToBox( auth_token, mural_id, sticky, box )
        box["top_free_space"] = new_y + sticky["height"] + 10
    print( "Done!" )

In [54]:
time.sleep(5)

# Quick!  After running this cell, switch to your browser 
# tab where the mural is to see the change
# ...

moveStickiesToClassBoxes( g_auth_token, g_mural_id, g_boxes_arr, g_stickies_arr )

Done!


Something like this:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-16_classify-by-exemplars_04.gif" width="50%" title="Image of a mural" />